# Feynman Reader

The goal of this project is to read in the Feynman lectures of theoretical physics in the form of HTML documents from https://www.feynmanlectures.caltech.edu/, and convert them into PDF documents.

In [64]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,os,sys,math,itertools

import pandas as pd
import plotly.express as px

import torch
import torch.nn as nn
from torch.nn import functional as F

from urllib.request import urlopen,urljoin
from bs4 import BeautifulSoup
import requests

from fpdf import FPDF 

from tqdm import tqdm

In [14]:
relative_storepath_mb = '../../../../../../../Books/FeynmanLectures/'

In [31]:
baseurl = 'https://www.feynmanlectures.caltech.edu/I_18.html'

In [113]:
pdf_path = relative_storepath_mb+'I_18/'
if not os.path.isdir(pdf_path): os.makedirs(pdf_path)

In [32]:
page = urlopen(baseurl)
soup = BeautifulSoup(page,'html.parser')

In [33]:
sent = soup.find_all('p')

In [52]:
first_idx = [k for k in range(len(sent)) if 'Editor, <i>The Feynman Lectures on Physics New Millennium Edition' in str(sent[k])][0]+1
sent = sent[first_idx:]

In [53]:
sent

[<p>(The photos for chapter 19, which includes the summary for this lecture, are missing from the Caltech Archives.)</p>,
 <p class="p">In the previous chapters we have been studying the mechanics
 of points, or small particles whose internal structure does not concern
 us. For the next few chapters we shall study the application of
 Newton’s laws to more complicated
 things. When the world becomes more complicated, it also becomes more
 interesting, and we shall find that the phenomena associated with the
 mechanics of a more complex object than just a point are really quite
 striking. Of course these phenomena involve nothing but combinations of
 Newton’s laws, but it is sometimes
 hard to believe that only $F = ma$ is at work.
 </p>,
 <p class="p">The more complicated objects we deal with can be of several kinds:
 water flowing, galaxies whirling, and so on. The simplest
 “complicated” object to analyze, at the start, is what we call a
 <em class="emph">rigid body</em>, a solid obje

In [107]:
url = baseurl
soup = BeautifulSoup(urlopen(baseurl),'html.parser')

urls = []
for img in tqdm(soup.find_all('img'),'Extracting images'):
    if 'data-src' in list(img.attrs.keys()):
        img_url = img.attrs.get('data-src')
    elif 'src' in list(img.attrs.keys()):
        img_url = img.attrs.get('src')
    else:
        continue
    if img_url=='': continue
    img_url = urljoin(url,img_url)
    #try:
    #    pos = img_url.index("?")
    #    img_url = img_url[:pos]
    #except ValueError:
    #    pass
    urls.append(img_url)

Extracting images: 100%|██████████| 14/14 [00:00<00:00, 55396.47it/s]


In [115]:
urls[0]

'https://www.feynmanlectures.caltech.edu/img/camera.svg'

In [114]:
img_path = relative_storepath_mb+'I_18/img/'
if not os.path.isdir(img_path): os.makedirs(img_path)

In [118]:
response = requests.get(urls[0],stream=True)

In [120]:
file_size = int(response.headers.get('Content-Length', 0))
filename = os.path.join(img_path, url.split('/')[-1])

In [123]:
filename

'../../../../../../../Books/FeynmanLectures/I_18/img/I_18.html'

In [124]:
progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
with open(filename, "wb") as f:
    for data in progress.iterable:
        f.write(data)
        progress.update(len(data))